# Homework 1 (Manoj Arulmurugan)

In [180]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("SCS")
Pkg.add("ECOS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Question 1: Model vs Instance

In [181]:
using JuMP
using GLPK

In [182]:
#Model:
model = Model(GLPK.Optimizer)

#Decision variables:
@variable(model, 0 <= x[1:3] <= 3) #d

#Objective function
@objective(model, Max, 6x[1] - 2x[2] + 3x[3]) #c

#Constraints:
@constraint(model, constr1, 2x[1] - x[2] + x[3] <= 2) #A,b

constr1 : 2 x[1] - x[2] + x[3] ≤ 2

#### 1a)

A = [2, -1, 1]
b = [2]
c = [6, -2, 3]
d = [3, 3, 3]

#### 1b)

ECOS: is optimized for conic problems, including second-order cone programming (SOCP) and some linear programming problems.
SCS: is designed for large-scale optimization problems, especially conic problems, using operator splitting.
GLPK: is a simplex and interior-point solver designed for linear programming (LP) and mixed-integer programming (MIP)

We use GLK here because The problem is a small-scale, straightforward linear programming problem. GLPK is specifically designed for LP and provides exact solutions.

#### 1c)

In [183]:
using JuMP
using GLPK

function LinearProgram(A, b, c, d)
    #=
    Parameters A, b, c, and d describe the linear program as
    described in the question.

    A: Matrix of constraint coefficients
    b: Vector of RHS values
    c: Vector of objective coefficients
    d: Vector of upper bounds on variables

    Returns: value of x (decision variables), objective value of m (model)
    =#

    n = length(c)
    #JuMP model
    m = Model(GLPK.Optimizer)

    #Adding decision variables with bounds
    @variable(m, 0 <= x[i=1:n] <= d[i])

    #Defining the objective function
    @objective(m, Max, sum(c[i] * x[i] for i in 1:n))

    #Adding constraints
    for i in 1:size(A, 1)
        @constraint(m, sum(A[i, j] * x[j] for j in 1:n) <= b[i])
    end

    optimize!(m)
    return value.(x), objective_value(m)
end

A = [2 -1 1]  #Coeffs of the constraints
b = [2]       #RHS of the constraints
c = [6, -2, 3] #Coeffs of the objective function
d = [3, 3, 3]  #Upper bounds

x, obj = LinearProgram(A, b, c, d)

# Display results
println()
println("Value of x1 equals $(x[1])")
println("Value of x2 equals $(x[2])")
println("Objective value equals $(obj)")
println()



Value of x1 equals 2.5
Value of x2 equals 3.0
Objective value equals 9.0



#### 1d)

In [184]:
# Input parameters
A = [1 1; 1 2]    # Coefficients of the constraints
b = [2; 3]        # RHS of the constraints
c = [2, 3]        # Coefficients of the objective function
d = [1.5, 1.25]   # Upper bounds on variables

# Solve the problem
x, obj = LinearProgram(A, b, c, d)

# Display results
println()
println("Value of x1 equals $(x[1])")
println("Value of x2 equals $(x[2])")
println("Objective value equals $(obj)")
println()


Value of x1 equals 1.0
Value of x2 equals 1.0
Objective value equals 5.0



## Question 2: Stigler Problem

#### 2b)

In [185]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("NamedArrays")
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [186]:
using JuMP
import CSV
import DataFrames
import HiGHS
import Test

In [187]:
# import Stigler's data set as a dataframe
raw = CSV.read("/Users/manoja/Downloads/stigler.csv", DataFrame)

(m,n) = size(raw)  # m = number of rows, n = number of columns

n_nutrients = 2:n      # indices of columns containing nutrients (skip the first one)

# the list of nutrients 
nutrients = names(raw)[n_nutrients]

9-element Vector{String}:
 "Calories (1000)"
 "Protein (g)"
 "Calcium (g)"
 "Iron (mg)"
 "Vitamin A (1000 IU)"
 "Thiamine (mg)"
 "Riboflavin (mg)"
 "Niacin (mg)"
 "Ascorbic Acid (mg)"

In [188]:
# the list of foods 
foods = raw[2:end,1]

77-element Vector{String31}:
 "Wheat Flour (Enriched)"
 "Macaroni"
 "Wheat Cereal (Enriched)"
 "Corn Flakes"
 "Corn Meal"
 "Hominy Grits"
 "Rice"
 "Rolled Oats"
 "White Bread (Enriched)"
 "Whole Wheat Bread"
 "Rye Bread"
 "Pound Cake"
 "Soda Crackers"
 ⋮
 "Raisins, Dried"
 "Peas, Dried"
 "Lima Beans, Dried"
 "Navy Beans, Dried"
 "Coffee"
 "Tea"
 "Cocoa"
 "Chocolate"
 "Sugar"
 "Corn Syrup"
 "Molasses"
 "Strawberry Preserves"

In [189]:
# minimum required amount of each nutrient
lower = Dict(zip(nutrients,raw[1,n_nutrients]))

Dict{String, Any} with 9 entries:
  "Calories (1000)"     => 3.0
  "Ascorbic Acid (mg)"  => 75
  "Calcium (g)"         => 0.8
  "Niacin (mg)"         => 18
  "Thiamine (mg)"       => 1.8
  "Iron (mg)"           => 12
  "Vitamin A (1000 IU)" => 5.0
  "Riboflavin (mg)"     => 2.7
  "Protein (g)"         => 70

In [190]:
# data[f,i] is the amount of nutrient i contained in food f
dataraw = Matrix(values(raw[2:end,2:end]))
data = NamedArray(dataraw,(foods,nutrients),("foods","nutrients"))
#@showall data

77×9 Named Matrix{Float64}
      foods ╲ nutrients │     Calories (1000)  …   Ascorbic Acid (mg)
────────────────────────┼────────────────────────────────────────────
Wheat Flour (Enriched)  │                44.7  …                  0.0
Macaroni                │                11.6                     0.0
Wheat Cereal (Enriched) │                11.8                     0.0
Corn Flakes             │                11.4                     0.0
Corn Meal               │                36.0                     0.0
Hominy Grits            │                28.6                     0.0
Rice                    │                21.2                     0.0
Rolled Oats             │                25.3                     0.0
White Bread (Enriched)  │                15.0                     0.0
Whole Wheat Bread       │                12.2                     0.0
Rye Bread               │                12.4                     0.0
⋮                                           ⋮  ⋱               

Data without the limits:

In [191]:
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [192]:
# Decision variables:
@variable(model, x[foods] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, String31["Wheat Flour (Enriched)", "Macaroni", "Wheat Cereal (Enriched)", "Corn Flakes", "Corn Meal", "Hominy Grits", "Rice", "Rolled Oats", "White Bread (Enriched)", "Whole Wheat Bread"  …  "Lima Beans, Dried", "Navy Beans, Dried", "Coffee", "Tea", "Cocoa", "Chocolate", "Sugar", "Corn Syrup", "Molasses", "Strawberry Preserves"]
And data, a 77-element Vector{VariableRef}:
 x[Wheat Flour (Enriched)]
 x[Macaroni]
 x[Wheat Cereal (Enriched)]
 x[Corn Flakes]
 x[Corn Meal]
 x[Hominy Grits]
 x[Rice]
 x[Rolled Oats]
 x[White Bread (Enriched)]
 x[Whole Wheat Bread]
 x[Rye Bread]
 x[Pound Cake]
 x[Soda Crackers]
 ⋮
 x[Raisins, Dried]
 x[Peas, Dried]
 x[Lima Beans, Dried]
 x[Navy Beans, Dried]
 x[Coffee]
 x[Tea]
 x[Cocoa]
 x[Chocolate]
 x[Sugar]
 x[Corn Syrup]
 x[Molasses]
 x[Strawberry Preserves]

In [193]:
#Objective:
@objective(model, Min, sum(x[f] for f in foods))

x[Wheat Flour (Enriched)] + x[Macaroni] + x[Wheat Cereal (Enriched)] + x[Corn Flakes] + x[Corn Meal] + x[Hominy Grits] + x[Rice] + x[Rolled Oats] + x[White Bread (Enriched)] + x[Whole Wheat Bread] + x[Rye Bread] + x[Pound Cake] + x[Soda Crackers] + x[Milk] + x[Evaporated Milk (can)] + x[Butter] + x[Oleomargarine] + x[Eggs] + x[Cheese (Cheddar)] + x[Cream] + x[Peanut Butter] + x[Mayonnaise] + x[Crisco] + x[Lard] + x[Sirloin Steak] + x[Round Steak] + x[Rib Roast] + x[Chuck Roast] + x[Plate] + x[Liver (Beef)] + [[...17 terms omitted...]] + x[Celery] + x[Lettuce] + x[Onions] + x[Potatoes] + x[Spinach] + x[Sweet Potatoes] + x[Peaches (can)] + x[Pears (can)] + x[Pineapple (can)] + x[Asparagus (can)] + x[Green Beans (can)] + x[Pork and Beans (can)] + x[Corn (can)] + x[Peas (can)] + x[Tomatoes (can)] + x[Tomato Soup (can)] + x[Peaches, Dried] + x[Prunes, Dried] + x[Raisins, Dried] + x[Peas, Dried] + x[Lima Beans, Dried] + x[Navy Beans, Dried] + x[Coffee] + x[Tea] + x[Cocoa] + x[Chocolate] + x[

In [194]:
# Constraints: Ensure nutrient requirements are met
for n in nutrients
    nutrient_column = data[:, n]  # Extract the column for the nutrient
    @constraint(model, sum(nutrient_column[f] * x[foods[f]] for f in 1:length(foods)) >= lower[n])
end

In [195]:
optimize!(model)

In [196]:
# Check the solution status
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found!")
    println("Cost: \$", objective_value(model))
    println("Food quantities:")
    for f in foods
        println(f, ": ", value(x[f]))
    end
else
    println("No optimal solution found.")
end

Optimal solution found!
Cost: $0.10866227820675686
Food quantities:
Wheat Flour (Enriched): 0.029519061676488274
Macaroni: 0.0
Wheat Cereal (Enriched): 0.0
Corn Flakes: 0.0
Corn Meal: 0.0
Hominy Grits: 0.0
Rice: 0.0
Rolled Oats: 0.0
White Bread (Enriched): 0.0
Whole Wheat Bread: 0.0
Rye Bread: 0.0
Pound Cake: 0.0
Soda Crackers: 0.0
Milk: 0.0
Evaporated Milk (can): 0.0
Butter: 0.0
Oleomargarine: 0.0
Eggs: 0.0
Cheese (Cheddar): 0.0
Cream: 0.0
Peanut Butter: 0.0
Mayonnaise: 0.0
Crisco: 0.0
Lard: 0.0
Sirloin Steak: 0.0
Round Steak: 0.0
Rib Roast: 0.0
Chuck Roast: 0.0
Plate: 0.0
Liver (Beef): 0.0018925572907052659
Leg of Lamb: 0.0
Lamb Chops (Rib): 0.0
Pork Chops: 0.0
Pork Loin Roast: 0.0
Bacon: 0.0
Ham, smoked: 0.0
Salt Pork: 0.0
Roasting Chicken: 0.0
Veal Cutlets: 0.0
Salmon, Pink (can): 0.0
Apples: 0.0
Bananas: 0.0
Lemons: 0.0
Oranges: 0.0
Green Beans: 0.0
Cabbage: 0.01121443524614487
Carrots: 0.0
Celery: 0.0
Lettuce: 0.0
Onions: 0.0
Potatoes: 0.0
Spinach: 0.0050076604667252025
Sweet Pot

-------

#### 2c)

**Analysis of the Results:**

The solution is feasible and meets all the nutrient requirements.

The diet costs 0.1087 dollars per day, or approximately 39.66 dollars per year (0.1087 * 365 = 39.66), which is very close to Stigler's reported cost of 39.93 dollars per year

Thus, the annual cost of the computed optimal diet is slightly lower than Stigler's value by 0.27 dollars per year. 

The foods making up my diet are: 
1) Wheat Flour (most prominent contributor)
2) Liver
3) Cabbage
4) Spinach
5) Navy Beans

#### 2d)

The cost of a vegetarian solution is likely to be higher than the cost of a non-vegetarian solution, beacuse:

* Vegetarians have limited Nutrient-Dense Options. Non-vegetarian foods, such as liver, are exceptionally nutrient-dense and provide high levels of certain key nutrients
* In a vegetarian diet, some nutrients like iron and vitamin B12, may be harder to source efficiently. Meeting these requirements may necessitate including more expensive or specialized vegetarian foods. This means the optimization algorithm may need to use multiple vegetarian foods in smaller quantities to meet the constraints, which could lead to higher costs.

A vegetarian solution will almost certainly have a higher cost than a non-vegetarian solution because it relies on a narrower and potentially less cost-efficient subset of foods to satisfy the same nutritional constraints.